In [1]:
import csv
import pandas as pd
import dask.dataframe as dd
import os

In [3]:
path = os.getcwd()

In [ ]:
tp = dd.read_csv(path + 'title.principals.tsv', sep = '\t')
tp = tp[['tconst', 'nconst']]
names = dd.read_csv(path + 'name.basics.tsv', sep = '\t')
names = names[['nconst', 'primaryName']]

In [ ]:
titles = pd.read_csv(path + 'title.basics.tsv', sep = '\t', error_bad_lines = False)
titles.head()
titles = titles[['tconst', 'primaryTitle', 'originalTitle', 'startYear']]
titles['tfix'] = titles['primaryTitle'].str.lower()
titles['tfix'] = titles['tfix'].str.replace('[^\w\s]','')

In [ ]:
wiki = pd.read_csv(path + 'wiki_movie_plots_deduped.csv')
wiki = wiki[['Release Year', 'Title', 'Plot']]
wiki['tfix'] = wiki['Title'].str.lower()
wiki['tfix'] = wiki['tfix'].str.replace('[^\w\s]','')
wiki.head()

In [ ]:
titles['startYear'] = titles['startYear'].astype(str)
wiki['Release Year'] = wiki['Release Year'].astype(str)
movies_main = titles.merge(wiki, left_on=['tfix', 'startYear'], right_on=['tfix', 'Release Year'])
movies_main.head()

In [ ]:
movies_main = dd.from_pandas(movies_main, npartitions=7)
movies_main = movies_main[['tconst', 'Title','startYear','Plot']]

In [ ]:
tp = tp.merge(movies_main[['tconst','Title']], on = 'tconst')

In [ ]:
names['primaryName'] = names['primaryName'].str.replace(' ','') + ' '

In [ ]:
tp = tp.merge(names, left_on='nconst', right_on='nconst')

In [ ]:
tp = tp.groupby(tp.tconst).primaryName.sum()
tp = tp.to_frame()
movies_main = movies_main.merge(tp, on = 'tconst')

In [ ]:
rev = dd.read_csv(path + 'title.ratings.tsv', sep = '\t')
movies_main = movies_main.merge(rev, on = 'tconst')

In [ ]:
movies_main = movies_main.compute()

In [ ]:
os.remove(path + "Movies_Reviews.csv")
movies_main.to_csv(path + 'Movies_Reviews.csv', index = False)